# `del`, `__del__` and garbage collection.

> I can't get my test to pass. `del X` doesn't work...

I think there's an aspect that often is omitted from python posts: garbage collection.

Let's say you're managing material that has to reside on disk - out of memory for example - but where it speeds things up massively by having a reference to data. Your code can then search using the python objects whilst the body of the data sits on disk:

In [ ]:
from itertools import count
from pathlib import Path
pages = count(1)
cachedir = Path('cache')  

class Table(object):
    def __init__(self, data):  # bytes
         self.page = cachedir / f"{next(pages)}.dat"
         with open(cache, 'wb') as fo:
             fo.write(data)

    def __del__(self):
        self.page.unlink()

And I'm using the code like this:

In [ ]:
t = Table(b'123')
# do something with the data.
del t

if you run this in a script, you'll see that the file isn't removed due to garbage collection being delayed by reference counts in the garbage collector pointing to the object. And then you ask yourself: "Why did I use `del` at all?

To see the real effect of your implementation using `del` to invoke `Table.__del__(self)` during debug may require the garbage collector to be called.

Here is how I chase down the bug:

### Step 1: check that your code is correct when called EXPLICITLY.


In [ ]:
t = Table(b'123')
path = str(t.page)
t.__del__()  # explicit call will work.
assert not Path(path).exists()

### Step 2: Check that your code is correct when invoked IMPLICITLY.


In [ ]:
t = Table(b'123')
path = str(t.page)
import gc
gc.collect()
del t
gc.collect()
assert not Path(path).exists()

### Step 3: Understand that this will not work if there are reference counts from the testing harness:


In [ ]:
t = Table(b'123')
path = str(t.page)
del t  # 
assert not Path(path).exists() # WILL PROBABLY RAISE!


### Step 4: Rewrite the test to accept that the test harness


In [ ]:

t = Table(b'123')

path = str(t.page)

del t

if path.exists():
   import gc
   gc.collect()
if path.exists():
   raise AttributeError()



My conclusion: Do read the [python datamodel](https://docs.python.org/3/reference/datamodel.html?highlight=__del__#object.__del__) !



